<a href="https://colab.research.google.com/github/LelouchFT/Traitement-du-texte-Python/blob/main/Chapitre-3-Nettoyage-Extraction-texte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans


#Lire le fichier de base dans une variable de texte brut
dfs = pd.read_excel("/content/data.xlsx", sheet_name=None)
df2019 = dfs['2019']

# Convertir les dates en format datetime et normaliser
df2019['Date de remplissage de la fiche'] = pd.to_datetime(df2019['Date de remplissage de la fiche'], errors='coerce').dt.normalize()

df2019['Date de remplissage de la fiche'] = df2019['Date de remplissage de la fiche'].fillna(df2019['Date de remplissage de la fiche'].mode()[0])
df2019['Date de naissance'] = pd.to_datetime(df2019['Date de naissance'], errors='coerce').dt.normalize()
df2019['Date de naissance'] = df2019['Date de naissance'].fillna(df2019['Date de naissance'].mode()[0])
df2019['Si oui preciser la date du dernier don.'] = pd.to_datetime(df2019['Si oui preciser la date du dernier don.'], errors='coerce').dt.normalize()
# Calculer l'âge s

df2019['Age'] = df2019.apply( lambda x: x['Date de remplissage de la fiche'].year - x['Date de naissance'].year ,
    axis=1
).astype('Int64')

# Afficher les 5 premières valeurs de la colonne Age
ages = df2019.pop('Age')
df2019.insert(1,'Age',ages)
df2 = df2019.copy()
#print(df2019[['Age','Date de remplissage de la fiche','Date de naissance']].head(15))
df2019 = df2019.drop('Date de naissance', axis =1)
df_vol = dfs['Volontaire']
df_vol = df_vol.drop(['ID','Horodateur'],axis = 1)
#df_vol['Date de remplissage de la fiche'] = pd.NaT
df_vol.insert(0,'Date de remplissage de la fiche'
,pd.NaT)
df_vol.columns = df2019.columns


df = pd.concat([df_vol,df2019],ignore_index = True)
df['Date de remplissage de la fiche'] = df['Date de remplissage de la fiche'].fillna(df['Date de remplissage de la fiche'].mode()[0])
imputer = SimpleImputer(strategy = 'constant',fill_value = 0)
df.columns = ['Date de remplissage de la fiche','Age',"Niveau d'etude", 'Genre', 'Taille', 'Poids',
       'Situation Matrimoniale (SM)', 'Profession',
       'Arrondissement de résidence', 'Quartier de Résidence', 'Nationalité',
       'Religion', 'A-t-il (elle) déjà donné le sang',
       'Si oui preciser la date du dernier don.', 'Taux d’hémoglobine',
       'ÉLIGIBILITÉ AU DON.',
       'Est sous anti-biothérapie',
       'Taux d’hémoglobine bas',
       'date de dernier Don < 3 mois',
       'IST récente (Exclu VIH, Hbs, Hcv)',
       'DDR',
       'DDR<14',
       'Allaitement',
       'A accoucher ces 6 derniers mois',
       'Interruption de grossesse  ces 06 derniers mois',
       'est enceinte',
       'Autre raisons ,preciser', 'Sélectionner "ok" pour envoyer',
       'Antécédent de transfusion',
       'Porteur(HIV,hbs,hcv)',
       'Opéré',
       'Drepanocytaire',       'Diabétique',
       'Hypertendus',
       'Asthmatiques',
       'Cardiaque',
       'Tatoué',
       'Scarifié',
       'Si autres raison preciser']


col = ['ÉLIGIBILITÉ AU DON.',
       'Est sous anti-biothérapie',
       'Taux d’hémoglobine bas',
       'date de dernier Don < 3 mois',
       'IST récente (Exclu VIH, Hbs, Hcv)',
       'DDR<14',
       'Allaitement',
       'A accoucher ces 6 derniers mois',
       'Interruption de grossesse  ces 06 derniers mois',
       'est enceinte',
       'Antécédent de transfusion',
       'Porteur(HIV,hbs,hcv)',
       'Opéré',
       'Drepanocytaire',
       'Diabétique',
       'Hypertendus',
       'Asthmatiques',
       'Cardiaque',
       'Tatoué',
       'Scarifié'
       ]
for c in col[1:] :
    df[c] = df[c].apply(lambda x : 1 if x == 'Oui' else 0 )
df['ÉLIGIBILITÉ AU DON.'] = df['ÉLIGIBILITÉ AU DON.'].apply(lambda x : 1 if x == 'Eligible' else 2 if x == 'Temporairement Non-eligible' else 0 )
df_cat = df[col]
col = df_cat.columns
df_cat = imputer.fit_transform(df_cat)
df_cat = pd.DataFrame(df_cat, columns = col)
df.head()

<ipython-input-68-fb88eb9dc3bf>:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2019['Date de remplissage de la fiche'] = pd.to_datetime(df2019['Date de remplissage de la fiche'], errors='coerce').dt.normalize()


,Date de remplissage de la fiche,Age,Niveau d'etude,Genre,Taille,Poids,Situation Matrimoniale (SM),Profession,Arrondissement de résidence,Quartier de Résidence,...,"Porteur(HIV,hbs,hcv)",Opéré,Drepanocytaire,Diabétique,Hypertendus,Asthmatiques,Cardiaque,Tatoué,Scarifié,Si autres raison preciser
0,2019-08-06,24,Universitaire,Homme,NaN,NaN,Célibataire,Etudiant (e),Douala 3,Yassa,...,0,0,0,0,0,0,0,0,0,NaN
1,2019-08-06,23,Universitaire,Homme,NaN,NaN,Célibataire,Sans Emplois,R A S,R A S,...,0,1,0,0,0,0,0,0,0,NaN
2,2019-08-06,41,Aucun,Homme,NaN,NaN,Marié (e),Tailleur,Douala 1,NEW BELL,...,0,0,0,0,0,0,0,0,0,Aucune information
3,2019-08-06,24,Universitaire,Homme,NaN,NaN,Célibataire,Etudiant,Douala 5,Ange raphael,...,0,0,0,0,0,0,0,0,0,NaN
4,2019-08-06,30,Secondaire,Homme,NaN,NaN,Célibataire,Militaire,Douala (Non précisé ),Beedi,...,0,0,0,0,0,0,0,0,0,NaN


In [ ]:
df.head()

,Date de remplissage de la fiche,Age,Niveau d'etude,Genre,Taille,Poids,Situation Matrimoniale (SM),Profession,Arrondissement de résidence,Quartier de Résidence,...,"Porteur(HIV,hbs,hcv)",Opéré,Drepanocytaire,Diabétique,Hypertendus,Asthmatiques,Cardiaque,Tatoué,Scarifié,Si autres raison preciser
0,2019-08-06,24,Universitaire,Homme,NaN,NaN,Célibataire,Etudiant (e),Douala 3,Yassa,...,0,0,0,0,0,0,0,0,0,NaN
1,2019-08-06,23,Universitaire,Homme,NaN,NaN,Célibataire,Sans Emplois,R A S,R A S,...,0,1,0,0,0,0,0,0,0,NaN
2,2019-08-06,41,Aucun,Homme,NaN,NaN,Marié (e),Tailleur,Douala 1,NEW BELL,...,0,0,0,0,0,0,0,0,0,Aucune information
3,2019-08-06,24,Universitaire,Homme,NaN,NaN,Célibataire,Etudiant,Douala 5,Ange raphael,...,0,0,0,0,0,0,0,0,0,NaN
4,2019-08-06,30,Secondaire,Homme,NaN,NaN,Célibataire,Militaire,Douala (Non précisé ),Beedi,...,0,0,0,0,0,0,0,0,0,NaN


In [13]:


df_autres = df.copy()
va = list(df2.columns)
df_autres = df_autres[['Date de remplissage de la fiche','Autre raisons ,preciser',  'Si autres raison preciser']]
#Extraction des tokens
#Autres raisons
df_autres['tokens indisponible']  =df_autres['Autre raisons ,preciser']
df_autres['tokens ineligible']  =df_autres['Si autres raison preciser']

In [ ]:

df_autres["tokens indisponible"].unique()

array([nan, 'Rapport non protégé et changement de partenaire ', 'Aucune',
       'RAISON NON PRECISEE', 'A été transfusé ',
       'Prise d’un médicament «\xa0DICLOJENAC\xa0»',
       'PAS DE RAISON SPECIFIQUES', 'RAPPORT NON PROTEGER',
       'RAPPORT NON PROTEGE', 'maladie cardio vasculaire',
       'Consommation de drogue ',
       'Eu à consommer de la cocaïne et d autre drogues ',
       'Changé de partenaire et eu des rapports non protégé', 'RAS',
       'RAPPORTS NON PROTEGES', 'CONSOMMATION DE DROGUES', 'opérée',
       'cocaïne', 'Pris de médicaments «\xa0DICLOJENAL\xa0»',
       'Voyagé dans des zones épidémiques '], dtype=object)

In [14]:
df_autres["tokens ineligible"].unique()

array([nan, 'Aucune information ',
       'EU UNE ENDOSCOPIE ( FIBROSCOPIE,  GASTROSCOPIE, COLOSCOPIE .......)',
       'physiquement amorti', 'TRAITE PAR ACUPUNCTURE', 'Mal de nerf',
       'PAS D INFORMATION SUR SON DOSSIER', 'Consommation de drogue ',
       'MENOPOSÉ', 'ETE TRAITE PAR ACUPUNCTURE', 'EU UNE ENDOSCOPIE',
       'DROGUES', 'maladie chronique, suivi medical regulier',
       'ANGINE DE LA POITRINE ,ARTÉRITE , INFARCTUS DU MYOCARDE '],
      dtype=object)

In [15]:

import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import FrenchStemmer

# Téléchargement des ressources nécessaires
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

import spacy
import re
import pandas as pd

# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

def preprocess_text(text):
    """ Nettoie et prépare une chaîne de texte pour le NLP """
    if pd.isna(text):  # Gérer les valeurs NaN
        return ""

    text = text.lower()  # Conversion en minuscules
    text = re.sub(r'\d+', '', text)  # Suppression des chiffres
    text = re.sub(r'[^\w\s]', '', text)  # Suppression des caractères spéciaux

    # Analyse du texte avec spaCy
    doc = nlp(text)

    # Tokenisation, suppression des stopwords et lemmatisation
    tokens = [token.lemma_ for token in doc if not token.is_stop]

    return " ".join(tokens)
def preprocess_dataframe(df, text_columns):
    """
    Applique le prétraitement NLP à un DataFrame.

    :param df: DataFrame contenant les textes
    :param text_columns: Liste des colonnes contenant du texte à prétraiter
    :return: DataFrame avec les colonnes nettoyées
    """
    df = df.copy()
    for col in text_columns:
        df[col] = df[col].apply(lambda x : x if x not in ( "Aucune information " , 'PAS DE RAISON SPECIFIQUES', 'Aucune','RAS','RAISON NON PRECISEE','PAS D INFORMATION SUR SON DOSSIER') else "")
        df[col] = df[col].fillna("").astype(str).apply(preprocess_text)
    df = df.dropna(thresh= 0.5)
    return df



df_pretraite = preprocess_dataframe(df_autres, ['tokens ineligible','tokens indisponible'])
print(df_pretraite[['tokens ineligible','tokens indisponible']].head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


  tokens ineligible tokens indisponible
0                                      
1                                      
2                                      
3                                      
4                                      


In [10]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 39.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#### Convertir en minuscules

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from flair.models import TextClassifier
from flair.data import Sentence

# Charger le modèle de classification de sentiment (XLM-RoBERTa pour plusieurs langues)
classifier = TextClassifier.load("xlm-roberta-large-sentiment")

# Exemple de texte en français
texte = "Ce produit est incroyable !"

# Créer une phrase
sentence = Sentence(texte)

# Appliquer le modèle de classification
classifier.predict(sentence)

# Afficher les résultats de sentiment
print(f"Sentiment: {sentence.labels}")

ModuleNotFoundError: No module named 'flair'

In [ ]:
pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from textblob import TextBlob

# Texte en français
texte = "L'intelligence artificielle est fascinante."

# Traduire en anglais
blob = TextBlob(texte)
texte_traduit = blob.translate(to="en")

print(f"Texte traduit : {texte_traduit}")

AttributeError: 'TextBlob' object has no attribute 'translate'

In [17]:
from textblob import TextBlob

# Texte en français
texte = "j aime ton l'intelligence ."

# Création d'un objet TextBlob
blob = TextBlob(texte)

# Analyse de sentiment
print(f"Polarité: {blob.sentiment.polarity}, Subjectivité: {blob.sentiment.subjectivity}")
df_autre1['sentiment indisponible'] = df_autres['tokens indisponible'].apply(lambda x : "positif" if TextBlob(x).sentiment.polarity >0 else (  "negative" if TextBlob(x).sentiment[0] < 0 else "neutre"))


df_autre['sentiment ineligible'] = df_autres['tokens ineligible'].apply(lambda x : "positif" if TextBlob(x).sentinent.polarity >0 else ( "negative" if TextBlob(x).sentiment[0] < 0 else "neutre"))

Polarité: 0.0, Subjectivité: 0.0


TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'float'>